In [113]:
%load_ext autoreload
%autoreload 2

import http_request
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [114]:
filepath = '/home/gqy/Desktop/Http_Gengerator/Version_2/'

In [115]:
grammar_filename = 'grammar.conf'
grammar = utils.load(filepath+grammar_filename)
grammar

{'End_Line': ['\r\n'],
 'Header_Line_CRLF': ['\r\n', '***'],
 'Request_Line': {'HTTP_Version': ['HTTP/1.1'],
  'Line_CRLF': ['\r\n'],
  'Method': ['GET', 'POST'],
  'Request_URI': ['/'],
  'Space': [' ']}}

In [116]:
rule_filename = 'rule.conf'
rule = utils.load(filepath+rule_filename)
rule

{'Header_Copy': [{'name': 'Host',
   'num': 2,
   'style': [-1, 1],
   'value': ['xxx123.tk', 'xxx456.tk']},
  {'name': 'Connection', 'num': 1, 'value': ['Keep-Alive']}]}

In [117]:
request = http_request.Request()
print (request.__dict__)

{'End_Line': [], 'Header_Lines': [], 'Headers': {}, 'Header_Line_CRLF': [], 'Request_Line': []}


In [118]:
def conf_parse(conf_file,request):
    '''
    解析带有配置选项的文件conf_file,并给request的各属性赋值,如:
    * request.Headers
    * request.Request_Line
    * request.Line_CRLF
    * request.End_Line
    '''
    
    conf_dict = utils.load(conf_file)
    '''
    conf_dict中每个key对应的value的type有三种取值情况:
    * value是字符串列表，如 "Header_Line_CRLF": ["\r\n"]
    * value是dict
    * value是dict的列表
    '''
    for key,values in conf_dict.items():
        func = getattr(request,key.lower())
        
        #key对应的值是dict
        if isinstance(values,dict):
            func(**values)
        #key对应的值是列表
        elif isinstance(values,list):
            for value in values:
                #列表中的元素是dict
                if isinstance(value,dict):
                    func(**value)
                #列表中的元素不是dict，可能是数字、str或list
                else:
                    #注意:这里不是程序解包，是位置参数
                    func(values)
                    break    

In [119]:
conf_parse(grammar,request)
print (request.__dict__)

{'End_Line': ['\r\n'], 'Header_Lines': [], 'Headers': {}, 'Header_Line_CRLF': ['\r\n', '***'], 'Request_Line': ['GET / HTTP/1.1\r\n', 'POST / HTTP/1.1\r\n']}


In [120]:
conf_parse(rule,request)
print (request.__dict__)

{'End_Line': ['\r\n'], 'Header_Lines': [], 'Headers': {'Connection': [(('Keep-Alive',), (0,))], 'Host': [(('xxx456.tk', 'xxx123.tk'), (-1, 1)), (('xxx456.tk', 'xxx123.tk'), (1, -1)), (('xxx123.tk', 'xxx456.tk'), (-1, 1)), (('xxx123.tk', 'xxx456.tk'), (1, -1))]}, 'Header_Line_CRLF': ['\r\n', '***'], 'Request_Line': ['GET / HTTP/1.1\r\n', 'POST / HTTP/1.1\r\n']}


In [122]:
#生成该request的所有字符串表示
requests = request.get_request()
for req in requests:
    print (req)

----------------> 16
GET / HTTP/1.1
Connection:Keep-Alive
 Host:xxx456.tk
Host: xxx123.tk


GET / HTTP/1.1
Connection:Keep-Alive
Host: xxx456.tk
 Host:xxx123.tk


GET / HTTP/1.1
Connection:Keep-Alive
 Host:xxx123.tk
Host: xxx456.tk


GET / HTTP/1.1
Connection:Keep-Alive
Host: xxx123.tk
 Host:xxx456.tk


GET / HTTP/1.1
Connection:Keep-Alive*** Host:xxx456.tk***Host: xxx123.tk***

GET / HTTP/1.1
Connection:Keep-Alive***Host: xxx456.tk*** Host:xxx123.tk***

GET / HTTP/1.1
Connection:Keep-Alive*** Host:xxx123.tk***Host: xxx456.tk***

GET / HTTP/1.1
Connection:Keep-Alive***Host: xxx123.tk*** Host:xxx456.tk***

POST / HTTP/1.1
Connection:Keep-Alive
 Host:xxx456.tk
Host: xxx123.tk


POST / HTTP/1.1
Connection:Keep-Alive
Host: xxx456.tk
 Host:xxx123.tk


POST / HTTP/1.1
Connection:Keep-Alive
 Host:xxx123.tk
Host: xxx456.tk


POST / HTTP/1.1
Connection:Keep-Alive
Host: xxx123.tk
 Host:xxx456.tk


POST / HTTP/1.1
Connection:Keep-Alive*** Host:xxx456.tk***Host: xxx123.tk***

POST / HTTP/1.1
Conne